<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a548cacdbd6d6506c5e546c76cf69dc88d98f452328418c1a4f60b5a088e5ee7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-24 11:45:49
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -55.65 K (-0.4%)
Current PnL: -19.51 L (-13.12%)
CY Booked + Current PnL: -8.07 L (-5.42%)
-------------------
Total profit:  1.40 L
Total loss:  -20.90 L
-------------------
Total Booked + Current PnL: 18.91 L (15.54%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.17%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.37 L (60.94%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.39,56.0,H-MC,3.76,193940.0,29270.0,14061.0,-1.25,17.77,7.25,26.31,89.0,2.08,1.38,27.18,X40N,NTT,AC
77,TTKPRESTIG,770.00,95.52,38.0,M-SC,2.91,84272.0,-16505.0,16602.0,-0.93,-16.38,19.70,0.09,245.0,-0.99,0.60,10.72,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.90,110770.0,10266.0,18842.0,0.08,10.21,17.01,28.96,79.0,0.54,0.79,47.65,MH,ATH,METALS
57,RELIANCE,1533.00,-13.29,50.0,H-LC,4.46,217448.0,7082.0,21701.0,0.29,3.37,9.98,13.68,37.0,0.33,1.55,20.45,XY25,NTT,REFINERIES
84,VOLTAS,1530.00,-1.78,46.0,H-MC,2.97,205950.0,14208.0,23540.0,0.02,7.41,11.43,19.69,99.0,0.60,1.47,14.71,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-14.17,73.0,H-SC,15.48,229760.0,-41001.0,370948.0,1.96,-15.14,161.45,121.86,133.0,-0.11,1.64,79.74,AR,ATH,IT
69,SURYODAY,240.00,51.32,66.0,H-SC,1.06,138873.0,-40198.0,105932.0,1.94,-22.45,76.28,36.71,135.0,-0.38,0.99,37.79,XR,NTT,BANKS
21,COFFEEDAY,80.00,-41.65,55.0,L-SC,42.87,86976.0,-26573.0,66624.0,1.77,-23.40,76.60,35.27,268.0,-0.40,0.62,112.88,XR,NTT,HOTELS
68,STARHEALTH,761.00,11.46,60.0,H-SC,4.84,248336.0,-32787.0,165640.0,1.25,-11.66,66.70,47.26,144.0,-0.20,1.77,33.05,XY24,NTT,INSURANCE
5,ASIANPAINT,3465.66,-13.84,44.0,H-LC,7.59,217439.0,-34329.0,87541.0,0.98,-13.64,40.26,21.13,27.0,-0.39,1.55,16.96,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1222.17,73.0,M-SC,9.87,179806.0,-1337.0,46804.0,-2.76,-0.74,26.03,25.10,235.0,-0.03,1.28,44.10,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.0,3732.43,55.0,M-SC,5.96,119534.0,-27834.0,79646.0,-2.71,-18.89,66.63,35.16,236.0,-0.35,0.85,22.09,XY24,NTT,AUTO
16,CAMPUS,393.0,-26.15,47.0,M-SC,0.39,153772.0,-14003.0,69843.0,-2.23,-8.35,45.42,33.28,210.0,-0.20,1.10,19.18,XY24,NTT,FOOTWEAR
26,GREENPANEL,537.0,211.37,48.0,M-SC,0.78,138885.0,-44193.0,123177.0,-2.13,-24.14,88.69,43.14,230.0,-0.36,0.99,30.64,XY24,NTT,MISC
8,AWL,485.0,-65.85,46.0,H-MC,3.39,243286.0,-58465.0,219882.0,-2.06,-19.38,90.38,53.50,116.0,-0.27,1.74,8.06,XY24,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1222.17,73.0,M-SC,9.87,179806.0,-1337.0,46804.0,-2.76,-0.74,26.03,25.10,235.0,-0.03,1.28,44.10,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.00,-21.68,41.0,H-MC,2.03,134673.0,-1115.0,43863.0,-0.82,-0.82,32.57,31.48,107.0,-0.03,0.96,11.20,X40,ATH,INSURANCE
23,DABUR,735.00,-7.80,38.0,H-MC,6.81,249508.0,1440.0,106964.0,-0.11,0.58,42.87,43.70,102.0,0.01,1.78,15.11,XY24,BTT,FMCG
52,PGHH,17973.08,-30.15,54.0,H-MC,3.67,202305.0,1485.0,67287.0,0.59,0.74,33.26,34.25,80.0,0.02,1.44,5.98,X40,ATH,FMCG
86,WIPRO,420.00,-13.79,41.0,M-LC,5.38,151881.0,936.0,108519.0,-1.87,0.62,71.45,72.51,53.0,0.01,1.08,6.68,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,51.0,H-SC,9.63,121905.0,-20574.0,40095.0,-0.13,-14.44,32.89,13.70,163.0,-0.51,0.87,43.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,95.52,38.0,M-SC,2.91,84272.0,-16505.0,16602.0,-0.93,-16.38,19.70,0.09,245.0,-0.99,0.60,10.72,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,51.0,H-SC,1.12,221454.0,-48213.0,84551.0,-0.06,-17.88,38.18,13.47,138.0,-0.57,1.58,12.01,XY24,NTT,PAINTS
66,SIS,528.0,2092.75,40.0,H-SC,4.69,88485.0,-22547.0,46154.0,0.52,-20.31,52.16,21.26,156.0,-0.49,0.63,19.55,OX40N,NTT,MISC
72,TATAELXSI,9161.0,-22.53,42.0,H-MC,6.90,104814.0,-27021.0,69251.0,-0.70,-20.50,66.07,32.03,98.0,-0.39,0.75,17.40,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,69.0,M-MC,9.74,235535.0,10573.0,157078.0,-0.38,4.70,66.69,74.53,192.0,0.07,1.68,38.47,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.39,56.0,H-MC,3.76,193940.0,29270.0,14061.0,-1.25,17.77,7.25,26.31,89.0,2.08,1.38,27.18,X40N,NTT,AC
84,VOLTAS,1530.00,-1.78,46.0,H-MC,2.97,205950.0,14208.0,23540.0,0.02,7.41,11.43,19.69,99.0,0.60,1.47,14.71,XY25,NTT,AC
17,CAMS,5211.76,-3.61,50.0,H-SC,2.27,109575.0,7571.0,36357.0,-1.82,7.42,33.18,43.06,122.0,0.21,0.78,25.81,X40N,ATH,MISC
23,DABUR,735.00,-7.80,38.0,H-MC,6.81,249508.0,1440.0,106964.0,-0.11,0.58,42.87,43.70,102.0,0.01,1.78,15.11,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.90,110770.0,10266.0,18842.0,0.08,10.21,17.01,28.96,79.0,0.54,0.79,47.65,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.93,43.0,M-SC,1.45,98168.0,6670.0,72085.0,-0.91,7.29,73.43,86.07,223.0,0.09,0.70,42.56,XR,NTT,DURABLES
11,BANKINDIA,190.00,-26.71,70.0,H-MC,7.47,188005.0,8197.0,104211.0,0.67,4.56,55.43,62.52,88.0,0.08,1.34,40.20,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.01,26.0,H-SC,10.70,125811.0,2475.0,131284.0,-0.72,2.01,104.35,108.45,119.0,0.02,0.90,24.81,AR,ATH,MISC
86,WIPRO,420.00,-13.79,41.0,M-LC,5.38,151881.0,936.0,108519.0,-1.87,0.62,71.45,72.51,53.0,0.01,1.08,6.68,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.89,32.0,H-LC,7.55,288631.0,-27211.0,139871.0,-1.33,-8.62,48.46,35.67,5.0,-0.19,2.06,4.05,X40N,ATH,FMCG
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
74,TCS,4476.75,-28.04,41.0,H-LC,12.12,286136.0,-59820.0,134684.0,-0.60,-17.29,47.07,21.64,1.0,-0.44,2.04,1.37,X40,ATH,IT
44,JIOFIN,387.00,-1.64,42.0,H-LC,13.27,267245.0,-120.0,65582.0,-0.58,-0.05,24.54,24.48,48.0,-0.00,1.91,54.90,XY24,BTT,FINANCE
30,HCLTECH,1943.91,-1.08,43.0,H-LC,9.91,226221.0,-15703.0,78974.0,0.78,-6.49,34.91,26.15,8.0,-0.20,1.62,6.90,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,47.0,H-LC,1.54,234788.0,-11363.0,90581.0,-0.85,-4.62,38.58,32.18,16.0,-0.13,1.68,29.76,X200,ATH,IT
42,ITC,452.0,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
65,SIEMENS,4671.5,0.17,52.0,H-LC,3.07,161055.0,-25040.0,72523.0,-0.60,-13.46,45.03,25.51,15.0,-0.35,1.15,18.69,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.29,50.0,H-LC,4.46,217448.0,7082.0,21701.0,0.29,3.37,9.98,13.68,37.0,0.33,1.55,20.45,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.39,47.0,H-LC,4.53,275863.0,10437.0,46345.0,0.90,3.93,16.80,21.40,11.0,0.23,1.97,11.30,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.04,41.0,H-LC,12.12,286136.0,-59820.0,134684.0,-0.60,-17.29,47.07,21.64,1.0,-0.44,2.04,1.37,X40,ATH,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
83,VBL,671.64,-19.89,32.0,H-LC,7.55,288631.0,-27211.0,139871.0,-1.33,-8.62,48.46,35.67,5.0,-0.19,2.06,4.05,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-11.80,57.0,H-LC,7.28,231374.0,-2770.0,51388.0,0.30,-1.18,22.21,20.77,86.0,-0.05,1.65,5.65,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,43.0,H-LC,9.91,226221.0,-15703.0,78974.0,0.78,-6.49,34.91,26.15,8.0,-0.20,1.62,6.90,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,52.0,H-LC,3.07,161055.0,-25040.0,72523.0,-0.60,-13.46,45.03,25.51,15.0,-0.35,1.15,18.69,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
19,CIPLA,1795.00,-21.15,44.0,H-LC,5.16,210240.0,5740.0,35678.0,0.49,2.81,16.97,20.25,10.0,0.16,1.50,11.79,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.84,44.0,H-LC,7.59,217439.0,-34329.0,87541.0,0.98,-13.64,40.26,21.13,27.0,-0.39,1.55,16.96,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,50.0,H-LC,4.46,217448.0,7082.0,21701.0,0.29,3.37,9.98,13.68,37.0,0.33,1.55,20.45,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.04,41.0,H-LC,12.12,286136.0,-59820.0,134684.0,-0.60,-17.29,47.07,21.64,1.0,-0.44,2.04,1.37,X40,ATH,IT
40,INFY,2275.00,-18.88,46.0,H-LC,8.15,316007.0,3119.0,168558.0,-0.93,1.00,53.34,54.87,3.0,0.02,2.26,7.71,X40,BTT,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
83,VBL,671.64,-19.89,32.0,H-LC,7.55,288631.0,-27211.0,139871.0,-1.33,-8.62,48.46,35.67,5.0,-0.19,2.06,4.05,X40N,ATH,FMCG
1,ABB,7934.00,-38.99,55.0,H-LC,8.61,255173.0,-6446.0,125647.0,0.33,-2.46,49.24,45.57,7.0,-0.05,1.82,8.58,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6894.44,57.0,L-SC,5.36,76755.0,-17055.0,93672.0,-0.74,-18.18,122.04,81.67,269.0,-0.18,0.55,48.64,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-7.36,39.0,H-SC,11.73,86262.0,-14701.0,109234.0,-0.90,-14.56,126.63,93.63,148.0,-0.13,0.62,28.49,SR,ATH,CHEMICALS
21,COFFEEDAY,80.00,-41.65,55.0,L-SC,42.87,86976.0,-26573.0,66624.0,1.77,-23.40,76.60,35.27,268.0,-0.40,0.62,112.88,XR,NTT,HOTELS
49,MASFIN,398.61,-17.34,50.0,H-SC,12.67,94170.0,-3810.0,25416.0,-0.16,-3.89,26.99,22.05,152.0,-0.15,0.67,36.88,XR,ATH,FINANCE
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.90,110770.0,10266.0,18842.0,0.08,10.21,17.01,28.96,79.0,0.54,0.79,47.65,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1222.17,73.0,M-SC,9.87,179806.0,-1337.0,46804.0,-2.76,-0.74,26.03,25.10,235.0,-0.03,1.28,44.10,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-14.17,73.0,H-SC,15.48,229760.0,-41001.0,370948.0,1.96,-15.14,161.45,121.86,133.0,-0.11,1.64,79.74,AR,ATH,IT
7,ATULAUTO,844.00,3732.43,55.0,M-SC,5.96,119534.0,-27834.0,79646.0,-2.71,-18.89,66.63,35.16,236.0,-0.35,0.85,22.09,XY24,NTT,AUTO
81,VAIBHAVGBL,521.00,60.33,61.0,H-SC,5.96,139133.0,-43642.0,154716.0,-0.19,-23.88,111.20,60.77,125.0,-0.28,0.99,24.62,XR,NTT,JEWELLERY
60,SAIL,228.00,49.16,69.0,M-MC,9.74,235535.0,10573.0,157078.0,-0.38,4.70,66.69,74.53,192.0,0.07,1.68,38.47,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.21
1,25,43.71
2,50,74.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.01
LC    32.43
MC    23.50
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.24
X40      14.71
X40N     11.37
XR       11.30
XY25     10.09
AR        8.45
OX40N     7.82
X200      1.68
MH        1.63
X5K       1.42
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.42
H-LC    25.98
H-MC    20.55
M-SC    13.06
M-LC     5.41
M-MC     2.65
L-SC     1.53
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.87,-5.65,40.51
IT,12.99,-15.64,78.69
FINANCE,10.06,-16.33,67.08
MISC,7.00,-17.32,82.55
BANKS,6.16,-15.39,76.09
PAINTS,5.64,-16.70,34.10
ELECTRICAL,5.49,-7.11,44.69
INSURANCE,3.72,-6.01,45.64
AC,3.55,2.69,13.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3142909.0,22
XR,1325641.0,14
AR,1148336.0,9
X40,787719.0,10
X40N,610646.0,9
OX40N,551491.0,10
XY25,426191.0,7
SR,249903.0,2
X5K,118860.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3671832.0,29
M-SC,1415483.0,17
H-MC,1226919.0,15
H-LC,1181205.0,15
M-LC,390927.0,4
M-MC,305685.0,2
L-SC,246801.0,3
L-MC,59916.0,1
L-LC,38345.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230917.0      6
M-SC       XY24       810609.0      7
H-SC       AR         785412.0      5
           XR         775389.0      7
H-MC       XY24       566568.0      4
H-LC       X40        494264.0      5
M-MC       XY24       305685.0      2
H-SC       X40N       304324.0      4
           OX40N      259893.0      4
           SR         249903.0      2
H-LC       X40N       226937.0      3
H-MC       X40        220490.0      4
H-LC       AR         198170.0      2
H-MC       XY25       168172.0      2
M-SC       AR         164754.0      2
M-LC       XY24       163548.0      2
L-SC       XR         160296.0      2
M-SC       OX40N      135842.0      4
M-LC       X5K        118860.0      1
M-SC       XR         117310.0      2
           XY25       114003.0      1
M-LC       XR         108519.0      1
H-LC       XY25       105671.0      3
H-MC       XR         104211.0      1
H-LC       X200        90581.0      1
L-SC       OX40N       86505.0      1
H-MC       X40N        79385.0      2
M-SC       X40         72965.0      1
H-MC       OX40N       69251.0      1
H-SC       MH          65994.0      1
H-LC       XY24        65582.0      1
L-MC       XR          59916.0      1
L-LC       XY25        38345.0      1
H-MC       MH          18842.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
